In [ ]:
from helper import *
import logging
from albumentations.pytorch import ToTensorV2
# from multiprocessing import Pool, set_start_method
import albumentations as A
import timm
from datetime import datetime
from itertools import cycle
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch import nn
# from torch.utils.tensorboard import SummaryWriter
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import matthews_corrcoef, accuracy_score
import os
import numpy as np
import pandas as pd
import random
import cv2
from math import ceil
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
CFG = {
    'seed': 42,
    'test_size': 1000,
    'lr': 1,
    'use_multi': False,
    'num_workers': 8,
    'batch_size': 64,
    'iterations': 1,
    'val_wait': 1,
    'scheduler_patience': 100,
    'saver_mode': 'all',
    'es_patience': 2,
    'rop_factor': 0.5,
    'rop_patience': 100,
}


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['seed'])

In [ ]:
%%time
df = pd.read_csv('./final_data2.csv')
df['bbox_endzone'] = df['bbox_endzone'].apply(process_bbox)
df['bbox_sideline'] = df['bbox_sideline'].apply(process_bbox)

In [ ]:
categorical_data_for_fitting = [
    ['home', 'CB', 'home', 'CB', ] ,
    ['home', 'DE', 'home', 'DE', ],
    ['home', 'FS', 'home', 'FS', ],
    ['home', 'TE', 'home', 'TE', ] ,
    ['home', 'ILB', 'home', 'ILB', ],
    ['home', 'OLB', 'home', 'OLB', ],
    ['home', 'T', 'home', 'T', ],
    ['home', 'G', 'home', 'G', ] ,
    ['home', 'C', 'home', 'C', ] ,
    ['home', 'QB', 'home', 'QB', ],
    ['home', 'WR', 'home', 'WR', ],
    ['home', 'RB', 'home', 'RB', ],
    ['home', 'NT', 'home', 'NT', ],
    ['home', 'DT', 'home', 'DT', ],
    ['home', 'MLB', 'home', 'MLB', ],
    ['home', 'SS', 'home', 'SS', ] ,
    ['home', 'OT', 'home', 'OT', ],
    ['home', 'LB', 'home', 'LB', ],
    ['home', 'OG', 'home', 'OG', ] ,
    ['home', 'SAF', 'home', 'SAF', ],
    ['home', 'DB', 'home', 'DB', ] ,
    ['home', 'LS', 'home', 'LS', ] ,
    ['home', 'K', 'home', 'K', ],
    ['home', 'P', 'home', 'P', ],
    ['home', 'FB', 'home', 'FB', ] ,
    ['home', 'S', 'home', 'S', ],
    ['home', 'DL', 'Ground', 'DL', ],
    ['away', 'HB', 'away', 'HB', ],
    ['away', 'HB', 'away', 'Ground', ],
]
    
one_hot = OneHotEncoder()
one_hot.fit(categorical_data_for_fitting)

In [ ]:
one_hot.transform(categorical_data_for_fitting).toarray()

a = np.array(categorical_data_for_fitting)
a.shape

In [ ]:
np.random.seed(CFG['seed'])
val_plays = np.random.choice(df['game_play'].unique(), size=2)
val_set = df.apply(lambda row: row[df['game_play'].isin(val_plays)]).reset_index()


In [ ]:
valid_aug = A.Compose([
        A.Normalize(mean=[0.], std=[1.]),
        ToTensorV2()
    ])

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, aug, one_hot_transform, feature_cols=['rel_pos_x',
                                                                 'rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
                                                                 'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x',
                                                                 'rel_acceleration_y', 'rel_acceleration_mag', 'rel_acceleration_ori',
                                                                 'G_flug', 'orientation_1', 'orientation_2']):

        self.df = df
        self.features = feature_cols
        self.aug = aug
        self.one_hot_transform = one_hot_transform

    def __len__(self):
        return len(self.df)

    def normalize_features(self, features):
        """
        normalizes the features of the players

       'rel_pos_x',
       'rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
       'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x',
       'rel_acceleration_y', 'rel_acceleration_mag', 'rel_acceleration_ori',
       'G_flug', 'orientation_1', 'orientation_2'
        """
        features /= 100
        features[3] /= 3.6
        features[7] /= 3.6
        features[11] /= 3.6
        features[13] /= 3.6
        features[14] /= 3.6
        return features

    def __getitem__(self, idx):
        window = 24
        frames_to_skip = 4

        row = self.df.iloc[idx]
        mid_frame = row['frame']

        label = float(row['contact'])
        imgs = []
        for view in ['Endzone', 'Sideline']:
            video = row['game_play'] + f'_{view}.mp4'
            frames = [mid_frame - window +
                      i for i in range(0, 2*window+1, frames_to_skip)]

            bbox_col = 'bbox_endzone' if view == 'Endzone' else 'bbox_sideline'
            bboxes = row[bbox_col][::frames_to_skip].astype(np.int32)

            if bboxes.sum() <= 0:
                imgs += [np.zeros((256, 256), dtype=np.float32)]*len(frames)
                continue

            for i, frame in enumerate(frames):
                img_new = np.zeros((256, 256), dtype=np.float32)
                cx, cy = bboxes[i]
                path = f'./work/train_frames/{video}_{frame:04d}.jpg'
                if os.path.isfile(path):
                    img_new = np.zeros((256, 256), dtype=np.float32)
                    if view == 'Endzone':
                        img = cv2.imread(path, 0)[
                            cy-76:cy+180, cx-128:cx+128].copy()
                        img_new[:img.shape[0], :img.shape[1]] = img
                    else:
                        img = cv2.imread(path, 0)[
                            cy-128:cy+128, cx-128:cx+128].copy()
                        img_new[:img.shape[0], :img.shape[1]] = img
                imgs.append(img_new)

        img = np.array(imgs).transpose(1, 2, 0)
        img = self.aug(image=img)["image"]

        features = np.array(row[self.features], dtype=np.float32)
        features[np.isnan(features)] = 0

        """
        rel_pos_x                0
        rel_pos_y                1
        rel_pos_mag              2
        rel_pos_ori              3
        rel_speed_x              4
        rel_speed_y              5
        rel_speed_mag            6
        rel_speed_ori            7
        rel_acceleration_x       8
        rel_acceleration_y       9
        rel_acceleration_mag     10
        rel_acceleration_ori     11 
        """
        if row['G_flug']:
            features[6] = row['speed_1']
            features[7] = row['direction_1']
            features[10] = row['acceleration_1']
            features[11] = row['direction_1']

            features[4] = row['speed_1']*np.sin(row['direction_1']*np.pi/180)
            features[5] = row['speed_1']*np.cos(row['direction_1']*np.pi/180)
            features[8] = row['acceleration_1'] * \
                np.sin(row['direction_1']*np.pi/180)
            features[9] = row['acceleration_1'] * \
                np.cos(row['direction_1']*np.pi/180)
        features = self.normalize_features(features)

        team_pos = np.array(
            row[['team_1', 'position_1', 'team_2', 'position_2']].fillna('Ground'))
        team_pos = self.one_hot_transform.transform(
            [team_pos]
        ).toarray()[0]

        return img, torch.from_numpy(np.hstack((features, team_pos)).astype(np.float32)), torch.as_tensor(label)

In [ ]:
test_s = MyDataset(val_set, aug=valid_aug, one_hot_transform=one_hot)

In [ ]:
img, f, l = test_s[0]

In [ ]:
l

In [ ]:
plt.imshow(img[0])
plt.show()

In [ ]:
class Validator():
    def __init__(self, test_df, aug, criterion, transform, verbose=True):
        self.test_set = MyDataset(
            test_df, aug=aug, one_hot_transform=transform)
        self.verbose = verbose
        self.test_loader = DataLoader(
            self.test_set, batch_size=CFG['batch_size'], num_workers=CFG['num_workers'], shuffle=False, pin_memory=False, persistent_workers=bool(CFG['num_workers']))
        self.criterion = criterion

    def validate(self, model):
        y_hat = []
        y = []
        model.eval()
        with torch.no_grad():
            for batch in self.test_loader:
                imgs, features, labels = batch

                imgs = imgs.to(0, non_blocking=True)
                features = features.to(0, non_blocking=True)
                labels = labels.to(0, non_blocking=True)

                preds = model(imgs, features)
                y.append(labels.cpu().detach().numpy())
                y_hat.append(preds.cpu().detach().numpy())

            y = np.hstack(y)
            y_hat = np.hstack(y_hat)
            y_hat = 1/(1+ np.exp(-y_hat))

            threshs = np.linspace(0.001, 0.999, 10000)

            mat_cors = []
            best = 0
            best_thresh = -1
            for thresh in threshs:
                _, val_mathew_corr,  _, = get_stats(
                    0.5, y, y_hat, cur_iter=f"Val", thresh=thresh)
                mat_cors.append(val_mathew_corr)
                if val_mathew_corr > best:
                    best = val_mathew_corr
                    best_thresh = thresh

        return mat_cors, threshs, best, best_thresh

In [ ]:
def get_stats(loss, y, y_pred, cur_iter='val', thresh=0.5):
    """
    Gets the stats for a particular batch
    """
    y_hat = (y_pred > thresh)*1.0
    mathew_corr = matthews_corrcoef(y, y_hat)
    acc = accuracy_score(y, y_hat)

    stats = f'Iteration: {cur_iter} || Loss: {loss:.5f} || mat_corr: {mathew_corr:.5f} || acc: {acc:.5f} || EOL'
    return stats.replace("\n", ""),  mathew_corr, acc

In [ ]:
args = {
            "test_df": val_set,
            "aug": valid_aug,
            "criterion": nn.BCELoss(),
            "transform": one_hot,
            "verbose": True
        }

valer = Validator(**args)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.backbone = timm.create_model(
            'resnet50', pretrained=True, num_classes=250, in_chans=26)
        self.mlp = nn.Sequential(
            nn.Linear(77, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.LayerNorm(64),
            nn.ReLU(),
            nn.Dropout(0.2),
        )
        self.fc = nn.Sequential(
            nn.Linear(64+250, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1),
        )

    def forward(self, img, feature):
        img = self.backbone(img)
        feature = self.mlp(feature)
        y = self.fc(torch.cat([img, feature], dim=1))
        return y.flatten()

In [ ]:
model = torch.load('./model_checkpoints/resnet50_v2_2023-02-14-00-29-19/best_model.pth')
model.eval()


In [ ]:
mat_cors, threshs, best, best_thresh = valer.validate(model)

In [ ]:
best, best_thresh

In [ ]:
plt.plot(threshs, mat_cors)
plt.show()

In [ ]:
1/(1+ np.exp(-0.51))